In [21]:
import sys

import spotipy
import spotipy.util as util
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

import json
from pprint import pprint

import pandas as pd

from configFM import CLIENT_ID, CLIENT_SECRET

In [22]:
#authenticating with client id & client secret credentials

credentials = oauth2.SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET)


token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

pprint(spotify)

C:\Users\Nada\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  
